In [17]:
import pandas as pd
import requests
import time
import zipfile
import io
import psycopg2

pd.set_option("display.max_rows", 400)

In [2]:
client_secret = "TfkrPQZCu9JJDJLNWtOqySbjZR7PK6VFoDhlst49eYVvzIlFvwDQnVuBBZHPdsZ8"
client_id = "6601caa9218359f1874ad95894cc5674"

## Testing Exports

In [ ]:
def startExport():
    surveyId = "SV_9Rfr1WtU3kfMXjw"
    url = f"https://iad1.qualtrics.com/API/v3/surveys/{surveyId}/export-responses"
    payload = {"format": "csv",
               "useLabels": true}
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "X-API-TOKEN": "egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF"
    }
    response = requests.post(url, json=payload, headers=headers)

    return response.json()

In [12]:
def checkExportStatus(progressId):
    surveyId = "SV_9Rfr1WtU3kfMXjw"
    httpStatus = "200 - OK"
    status = ""
    percentComplete = ""
    url = f"https://iad1.qualtrics.com/API/v3/surveys/{surveyId}/export-responses/{progressId}"
    while ((httpStatus == "200 - OK") and (percentComplete != 100) and (status != "failed")):
        headers = {
            "Accept": "application/json",
            "X-API-TOKEN": "egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF"
        }

        response = requests.get(url, headers=headers)
        re = response.json()

        httpStatus = re["meta"]["httpStatus"]
        status = re["result"]["status"]
        percentComplete = re["result"]["percentComplete"]

        time.sleep(1)

    if((percentComplete == 100) and (status == "complete")):
        return re["result"]["fileId"]
    else:
        return None

In [22]:
def getFile(fileId):
    surveyId = "SV_9Rfr1WtU3kfMXjw"
    url = f"https://iad1.qualtrics.com/API/v3/surveys/{surveyId}/export-responses/{fileId}/file"
    headers = {
        "Accept": "application/octet-stream, application/json",
        "X-API-TOKEN": "egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF"
    }
    response = requests.get(url, headers=headers)
    return response

## Response Functions

In [3]:
def getResponse(surveyId) :
    url = f"https://iad1.qualtrics.com/API/v3/surveys/{surveyId}/export-responses"
    payload = {"format": "csv",
               "useLabels": True}
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "X-API-TOKEN": "egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF"
    }
    response = (requests.post(url, json=payload, headers=headers)).json()

    if (response["result"]["progressId"] and (response["meta"]["httpStatus"] == "200 - OK")):
        httpStatus = "200 - OK"
        status= ""
        percentComplete = ""
        exportProgressId = response["result"]["progressId"]
        print("File Export Started")
        print("---")
        url = f"https://iad1.qualtrics.com/API/v3/surveys/{surveyId}/export-responses/{exportProgressId}"
        while ((httpStatus == "200 - OK") and (percentComplete != 100) and (status != "failed")):
            headers = {
                "Accept": "application/json",
                "X-API-TOKEN": "egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF"
            }

            response = (requests.get(url, headers=headers)).json()

            httpStatus = response["meta"]["httpStatus"]
            status = response["result"]["status"]
            percentComplete = response["result"]["percentComplete"]
            print(f"File Export {percentComplete}% complete")
            print("---")
            time.sleep(2)

        if((percentComplete == 100) and (status == "complete")):
            print("File Export Complete")
            print("---")
            fileId = response["result"]["fileId"]
            url = f"https://iad1.qualtrics.com/API/v3/surveys/{surveyId}/export-responses/{fileId}/file"
            headers = {
                "Accept": "application/octet-stream, application/json",
                "X-API-TOKEN": "egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF"
            }
            response = requests.get(url, headers=headers)
            if (response.status_code == 200):
                print("File successfully retrieved!")
                return response
            else:
                print(f"Failed to retrieve file. Error: {response.status_code}")
                print("---------------------------------------")
                return None
        else:
            print(f'Export failed:\nhttpStatus: {response["meta"]["httpStatus"]}\npercentComplete: {response["result"]["percentComplete"]}\nStatus: {response["result"]["status"]}')

            return None
    else:
        print(f"Export unable to start. Error: {response.status_code}")
        return None

In [4]:
def processOldFile(file):
    try:
        with zipfile.ZipFile(io.BytesIO(file.content)) as z:
            z.extractall('./qualtricsCSV/')
            fileName = z.namelist()
            rawSurvey = pd.read_csv('./qualtricsCSV/' + fileName[0])
    except(zipfile.BadZipFile, IndexError, pd.errors.EmptyDataError, pd.errors.ParserError, KeyError) as e:
        print(f"Error processing file: {e}")
        return None, None

    print(f"File {fileName} extracted successfully")
    print("---")
    filteredSurvey = rawSurvey[rawSurvey.columns[18:25]]
    filteredSurvey.columns = ["Email", "FirstName", "LastName", "UAPosition", "UADeptOther", "UADept", "OtherDept"]
    filteredSurvey = filteredSurvey.drop([0,1])
    filteredSurvey = filteredSurvey.dropna(subset=["Email"])
    filteredSurvey = filteredSurvey.drop_duplicates(subset=["Email"])
    pd.set_option("display.max_rows", 100)
    filteredSurvey["Series"] = fileName[0][12:-18]

    searchFor = ["Graduate student", "PostDoc", "Undergraduate student", "Faculty", "Staff"]

    UA = filteredSurvey.loc[(filteredSurvey["Email"].str.contains("arizona.edu")) | (filteredSurvey["UAPosition"].str.contains("|".join(searchFor)))]
    nonUA = pd.concat([filteredSurvey, UA]).drop_duplicates(keep=False)

    UA = UA.reset_index(drop=True)
    nonUA = nonUA.reset_index(drop=True)

    UA_Filtered = UA[["Email", "FirstName", "LastName", "Series"]]
    nonUA_Filtered = nonUA[["Email", "FirstName", "LastName", "UADeptOther", "OtherDept", "Series"]]

    print(f"File {fileName} processed successfully")
    print("---------------------------------------")

    return UA_Filtered, nonUA_Filtered


In [19]:
def processSpecialFile(file):
    try:
        with zipfile.ZipFile(io.BytesIO(file.content)) as z:
            z.extractall()
            fileName = z.namelist()
            rawSurvey = pd.read_csv(fileName[0])
    except(zipfile.BadZipFile, IndexError, pd.errors.EmptyDataError, pd.errors.ParserError, KeyError) as e:
        print(f"Error processing file: {e}")
        return None, None

    print(f"File {fileName} extracted successfully")
    print("---")
    filteredSurvey = rawSurvey[rawSurvey.columns[17:24]]
    filteredSurvey.columns = ["FirstName", "LastName", "Email", "UAPosition", "UADeptOther", "UADept", "OtherDept"]
    filteredSurvey = filteredSurvey.drop([0,1])
    filteredSurvey = filteredSurvey.dropna(subset=["Email"])
    filteredSurvey = filteredSurvey.drop_duplicates(subset=["Email"])
    filteredSurvey["Series"] = fileName[0][12:-4]

    searchFor = ["Graduate student", "PostDoc", "Undergraduate student", "Faculty", "Staff"]

    UA = filteredSurvey.loc[(filteredSurvey["Email"].str.contains("arizona.edu")) | (filteredSurvey["UAPosition"].str.contains("|".join(searchFor)))]
    nonUA = pd.concat([filteredSurvey, UA]).drop_duplicates(keep=False)

    UA = UA.reset_index(drop=True)
    nonUA = nonUA.reset_index(drop=True)

    UA_Filtered = UA[["Email", "FirstName", "LastName", "Series"]]
    nonUA_Filtered = nonUA[["Email", "FirstName", "LastName", "UADeptOther", "OtherDept", "Series"]]

    print(f"File {fileName} processed successfully")
    print("---------------------------------------")

    return UA_Filtered, nonUA_Filtered

In [21]:
def processFile(file):
    try:
        with zipfile.ZipFile(io.BytesIO(file.content)) as z:
            z.extractall()
            fileName = z.namelist()
            rawSurvey = pd.read_csv(fileName[0])
    except(zipfile.BadZipFile, IndexError, pd.errors.EmptyDataError, pd.errors.ParserError, KeyError) as e:
        print(f"Error processing file: {e}")
        return None, None

    print(f"File {fileName} extracted successfully")
    print("---")
    filteredSurvey = rawSurvey[rawSurvey.columns[18:24]]
    filteredSurvey.columns = ["UA_Afilliated", "FirstName", "LastName", "UAEmail", "Organization", "Non_UA_Email", "Workshops"]
    filteredSurvey = filteredSurvey.drop([0,1])
    filteredSurvey = filteredSurvey.dropna(subset=["Email"])
    filteredSurvey = filteredSurvey.drop_duplicates(subset=["Email"])
    filteredSurvey["Series"] = fileName[0][12:-18]

    UA = filteredSurvey.dropna(subset=["UAEmail"])
    nonUA = filteredSurvey.dropna(subset=["Non_UA_Email"])

    UA = UA.reset_index(drop=True)
    nonUA = nonUA.reset_index(drop=True)

    UA_Filtered = UA[["Email", "FirstName", "LastName", "Workshops", "Series"]]
    nonUA_Filtered = nonUA[["Non_UA_Email", "FirstName", "LastName", "Organization", "Workshops", "Series"]]

    print(f"File {fileName} processed successfully")
    print("---------------------------------------")

    return UA_Filtered, nonUA_Filtered

In [30]:
seriesSurveyIDs = ["SV_5BjjU8Exp3GS2sS", "SV_9mLQalGxqyBb7mK", "SV_0fbG5COlxNj5phI", "SV_9TDHtSJnP6d32gS", "SV_exLckAyzfnUTKHc", "SV_9Rfr1WtU3kfMXjw"]

UA = pd.DataFrame()
nonUA = pd.DataFrame()

for id in seriesSurveyIDs:
    print(f"Starting export for surveyId: {id}")
    print("------------------------------------------------")
    file = getResponse(id)
    print("------------------------------------------------\n")

    print(f"Starting processing for surveyId: {id}")
    print("------------------------------------------------")
    tempUA, tempNonUA = processOldFile(file)
    UA = pd.concat([UA, tempUA])
    nonUA = pd.concat([nonUA, tempNonUA])

    print(f"Survey {id} Complete!")
    print("------------------------------------------------\n")

file = getResponse("SV_9oGXsP5SKL9jNRA")
tempUA, tempnonUA = processSpecialFile(file)
tempUA['Series'] = 'Cracking the Coding Interview'
tempnonUA['Series']  = 'Cracking the Coding Interview'
UA = pd.concat([UA, tempUA])
nonUA = pd.concat([nonUA, tempNonUA])

Starting export for surveyId: SV_5BjjU8Exp3GS2sS
------------------------------------------------
File Export Started
---
File Export 0.0% complete
---
File Export 0.0% complete
---
File Export 100.0% complete
---
File Export Complete
---
File successfully retrieved!
------------------------------------------------

Starting processing for surveyId: SV_5BjjU8Exp3GS2sS
------------------------------------------------
File ['UA DataLab: Classical Machine Learning (Spring 2024).csv'] extracted successfully
---
File ['UA DataLab: Classical Machine Learning (Spring 2024).csv'] processed successfully
---------------------------------------
Survey SV_5BjjU8Exp3GS2sS Complete!
------------------------------------------------

Starting export for surveyId: SV_9mLQalGxqyBb7mK
------------------------------------------------
File Export Started
---
File Export 0.0% complete
---
File Export 0.0% complete
---
File Export 100.0% complete
---
File Export Complete
---
File successfully retrieved!
----

In [12]:
UA['Series'] = UA['Series'].str.replace('Cracking the co', 'Cracking The Coding Interview')
nonUA['Series'] = nonUA['Series'].str.replace('Cracking the co', 'Cracking The Coding Interview')

In [14]:
def createRegistreeWorkshop(row, regID, conn, cur):
    cur.execute("""
                SELECT workshopID FROM workshops WHERE seriesID = (SELECT seriesID FROM series WHERE seriesName = %s)
                """, (row.iloc[7],))
    conn.commit()
    workshops = cur.fetchall()
    for ID in workshops:
        workshopID = ID[0]
        cur.execute("""
                        INSERT INTO registreeworkshops (RegID, WorkshopID, Registered)
                        VALUES (%s, %s, %s)
                        ON CONFLICT DO NOTHING;
                    """, (regID, workshopID, True))
        conn.commit()

def uploadRegistrees(UA, conn, cur):
    for _, row in UA.iterrows():
        #Call my custom function hash which will create a hash from the First Name, Last Name, Email, and Major
        print(row.iloc[3])
        cur.callproc('hashRegistree', (row.iloc[3],))
        hashedNum = cur.fetchone()[0]
        print(hashedNum)
        cur.execute("""
            INSERT INTO registreeInfo (RegID, FirstName, LastName, NetID, Email, College, Department, Major)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT DO NOTHING;
        """, (hashedNum, row.iloc[0], row.iloc[1], row.iloc[2], row.iloc[3], row.iloc[4], row.iloc[5], row.iloc[6]))
        conn.commit()

        createRegistreeWorkshop(row, hashedNum, conn, cur)     

In [35]:
testUA = UA.copy()
testnonUA = nonUA.copy()
testUA['NetID'] = None
testUA['College'] = None
testUA['Department'] = None
testUA['Major'] = None
testUA = testUA[['FirstName', 'LastName', 'NetID', 'Email', 'College', 'Department', 'Major', 'Series']]

testnonUA['Major'] = None
testnonUA['NetID'] = None
testnonUA.columns = ['Email', 'FirstName', 'LastName', 'College', 'Department', 'Series', 'Major', 'NetID',]
testnonUA = testnonUA[['FirstName', 'LastName', 'NetID', 'Email', 'College', 'Department', 'Major', 'Series']]

In [38]:
conn = psycopg2.connect(database = "DataLab", 
                        user = "postgres", 
                        host= 'localhost',
                        password = "",
                        port = 5432)
cur = conn.cursor()


uploadRegistrees(testUA, conn, cur)
uploadRegistrees(testnonUA, conn, cur)

cur.close()
conn.close()

cosi@arizona.edu
4616343622010110217
barrette@arizona.edu
7521418346036733054
kashyapnadendla@arizona.edu
-2934687906461763499
ayeshakhatun@arizona.edu
3426014083416905360
tianyili1@gmail.com
7714370274836070474
nferrante@arizona.edu
3727118599387067309
praveenkpappala@arizona.edu
-5203727546136622884
tianyili1@arizona.edu
4020877818367534995
wangyw@arizona.edu
-4474409817818578833
keshavbhusal@arizona.edu
8552476225432232823
pansemanas0@gmail.com
250677359474826468
gowthamg@arizona.edu
-5926005403281659340
Ianstewart@arizona.edu
-5299876257453479414
katiechambe@arizona.edu
6227843774926749725
sjaganathan@arizona.edu
4937023485114215832
riyanshibohra@arizona.edu
1234280771741347413
dongchen1@arizona.edu
-7687827853389264025
shashwatsingh1@arizona.edu
1166673701365963990
derekrice@arizona.edu
3189282454521313177
kylimcqueen@arizona.edu
-8013976643527542668
praneethay@arizona.edu
3606774004140026367
tiruthani@arizona.edu
-6663500048009865107
wlavoy@arizona.edu
-5879748388356937247
mkrish